<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments_Feb23/Model-5-keras_lstmConv_clr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip
import keras.backend as K

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D, Input, SpatialDropout1D, Embedding, Dense, Activation, Bidirectional, LSTM,CuDNNLSTM, CuDNNGRU, GlobalMaxPooling1D,GlobalAveragePooling1D, concatenate, Dropout
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.engine import Layer
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
best_model.hdf5			   test.csv
ext_fasttextwiki_embeddings.npy    test_features.npy
ext_glove_embeddings.npy	   test_labels.csv
ext_lexvec_embeddings.npy	   toxic_lookup_en_wiki_fasttext_300d.npy
features.npy			   train.csv
inputed_lexvec_embeddings.npy	   word_index.npy
local_fasttextwiki_embeddings.npy  x_test.npy
Model-runs			   x_train.npy
sample_submission.csv		   y_train.npy
submission_baseline_lr.csv


In [0]:
#configs
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector) ##sum([word[1] for word in vocab[:120000]])=92%##
maxlen = 250 # max number of words in a question to use ## train.num_words.quantile(q=0.96, interpolation='nearest')=258 ##
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits

SEED = 2019
def seed_everything(seed=2019):
    random.seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()


In [0]:
os.chdir("/content/drive/My Drive/kaggle_toxic_comments/")
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
features = np.load("features.npy")
test_features = np.load("test_features.npy")
word_index = np.load("word_index.npy").item()
ext_lexvec_embeddings = np.load("ext_lexvec_embeddings.npy")

In [0]:
# Train simple LSTM using different embeddings
# Top compeitions- QIQC and Toxic comments
# ensembles (stacking and blending)
# Pretrained models- ULMFit and BERT
# Gensim and fasttext text classification
# Other SOTAs

In [0]:
##Model-3: https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/80568
# One cycle learning
# 10 fold CV with 10 model runs with different initializers/seeds/embeddings
# Model with different embeddings and order of rows
# Extra features
# Pseudo labeling
# use focal loss

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.1)

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

            
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):     
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v) 
        K.set_value(self.model.optimizer.lr, self.clr())

In [20]:
inp = Input(shape=(maxlen,))
x   = Embedding(max_features, embed_size, weights=[ext_lexvec_embeddings], trainable=False)(inp)
x   = SpatialDropout1D(rate=0.2)(x)
x   = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
x   = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x)
x1  = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x1   = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x1)
x_max   = GlobalMaxPooling1D()(x)
x_avg   = GlobalAveragePooling1D()(x)
x1_max  = GlobalMaxPooling1D()(x1)
x1_avg  = GlobalAveragePooling1D()(x1)
c   = concatenate([x_max,x_avg,x1_max,x1_avg])
y   = Dense(64, activation='relu')(c)
y   = Dropout(0.2)(y)
out = Dense(6, activation='sigmoid')(y)
model=Model(inputs=inp,outputs=out)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 250, 300)     36000000    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 250, 300)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 250, 256)     330240      spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv1d_3 (

In [27]:
# for one cycle policy: step_size=(len(X_train)/batch_size)*n_ecpochs/2
clr = CyclicLR(base_lr=1e-6, max_lr=1e-3,step_size=5610., mode='exp_range',gamma=0.99994)
model.fit(X_train, Y_train, batch_size = 128, epochs = 10, validation_data = (X_valid, Y_valid), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])


Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 291s 2ms/step - loss: 0.0425 - acc: 0.9836 - val_loss: 0.0440 - val_acc: 0.9830

 ROC-AUC - epoch: 1 - score: 0.982293

Epoch 00001: val_loss did not improve from 0.04151
Epoch 2/5
143613/143613 [==============================] - 290s 2ms/step - loss: 0.0435 - acc: 0.9833 - val_loss: 0.0423 - val_acc: 0.9835

 ROC-AUC - epoch: 2 - score: 0.984566

Epoch 00002: val_loss did not improve from 0.04151
Epoch 3/5
143613/143613 [==============================] - 290s 2ms/step - loss: 0.0426 - acc: 0.9834 - val_loss: 0.0408 - val_acc: 0.9842

 ROC-AUC - epoch: 3 - score: 0.986356

Epoch 00003: val_loss improved from 0.04151 to 0.04081, saving model to best_model.hdf5
Epoch 4/5
143613/143613 [==============================] - 290s 2ms/step - loss: 0.0397 - acc: 0.9842 - val_loss: 0.0397 - val_acc: 0.9845

 ROC-AUC - epoch: 4 - score: 0.988139

Epoch 00004: val_loss improved from 0.04081

In [28]:
pred = model.predict(x_test, batch_size = 1024, verbose = 1)

153164/153164 [==============================] - 49s 323us/step


In [0]:
submission = pd.read_csv("./sample_submission.csv")
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = (pred)
submission.to_csv("submission_model5_Feb23_auc_988317.csv", index = False)

In [0]:
sub1 = pd.read_csv("/content/drive/My Drive/kaggle_toxic_comments/Model-runs/submission_model5_Feb23_auc_988317.csv")
sub2 = pd.read_csv("/content/drive/My Drive/kaggle_toxic_comments/Model-runs/submission_model3_Feb23_auc_988480.csv")
df = pd.concat([sub1, sub2]).groupby(level=0).mean()
df['id'] = sub1['id']
df = df[["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
df.to_csv("submission_model5_Feb23_ensemble_988317+988480.csv", index = False)


In [45]:
df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999421,4.815847e-01,0.977320,8.526233e-02,0.939398,0.382825
1,0000247867823ef7,0.000972,2.682209e-07,0.000053,3.576279e-07,0.000098,0.000005
2,00013b17ad220c46,0.000403,2.086163e-07,0.000065,4.321337e-07,0.000028,0.000003
3,00017563c3f7919a,0.000503,8.940697e-08,0.000017,5.811453e-07,0.000038,0.000003
4,00017695ad8997eb,0.000981,8.940697e-08,0.000028,1.296401e-06,0.000062,0.000003
